In [5]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm.autonotebook import tqdm

c:\Users\dell\Documents\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [7]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="89c78740-590d-4585-861f-0492769c1e93")

In [8]:
pc.create_index(
    name="ezra-chatbot",
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [9]:
def load_pdf(data):
    loader=DirectoryLoader(data,
                glob="*.pdf",
                loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [10]:
extracted_data=load_pdf("data/")

In [11]:
def text_split(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [12]:
text_chunks =text_split(extracted_data)

In [13]:
def download_hf_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [14]:
embeddings=download_hf_embeddings()

c:\Users\dell\Documents\anaconda3\envs\mchatbot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
import os

os.environ["PINECONE_API_KEY"] = "89c78740-590d-4585-861f-0492769c1e93"

In [17]:
from langchain_pinecone import PineconeVectorStore


index_name="ezra-chatbot"

docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings, index_name=index_name)

In [18]:
docsearch=PineconeVectorStore.from_existing_index(index_name,embeddings)

query="\ "

docs=docsearch.similarity_search(query,k=3)

print("result",docs)

result [Document(metadata={'page': 275.0, 'source': 'data\\Medical_book.pdf'}, page_content='longer periods of time.'), Document(metadata={'page': 364.0, 'source': 'data\\Medical_book.pdf'}, page_content='out ofdirect light.'), Document(metadata={'page': 558.0, 'source': 'data\\Medical_book.pdf'}, page_content='is lessof a problem.')]


In [19]:
prompt_template="""
Use the following pieces of information to answer the user's question.
return the answer appropriate to the question. don't reply wrong answer 

Context:{context}
Question:{question}

only return helpfull answer below and nothing else
helpfull answer:

"""

In [20]:
PROMPT=PromptTemplate(template=prompt_template,input_variables=["context","question"])
chain_type_kwargs={"prompt":PROMPT}

In [21]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                model_type="llama",
                config={'max_new_tokens':512,
                        'temperature':0.5})

In [22]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

In [23]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa.invoke({"query": user_input})
    print("Response: ", result["result"])

Response:  Fever is a temporary increase in body temperature, usually caused by an infection or inflammation somewhere in the body. It is often accompanied by other symptoms such as shivering, sweating, headache, muscle aches, and fatigue. Fever can be treated with medications such as acetaminophen or ibuprofen, which can help reduce the body temperature and relieve symptoms. In severe cases, hospitalization may be necessary to manage the fever and other symptoms.


: 